# Inspecter les meilleures stratégies

Ce notebook charge les résultats de walk-forward / grid-search (`wf_results.json`, `wf_top5.csv`) et permet d'inspecter visuellement les courbes d'équity et les points d'entrée / sortie des meilleures stratégies. Si aucun résultat n'est présent, le notebook génère des données synthétiques et exécute une petite grille d'exemple.

In [ ]:
# Cell: imports
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# imports from the project
from src.bot.strategies import MovingAverageStrategy
from src.bot.backtest import Backtester
from src.bot.optimizer import grid_search_ma, get_top_results

# Inline plotting (works in Jupyter)
%matplotlib inline

print('Imports OK')


In [ ]:
# Helper functions

def load_walkforward_json(path):
    p = Path(path)
    if not p.exists():
        return None
    with open(p, 'r') as f:
        return json.load(f)


def load_top_csv(path):
    p = Path(path)
    if not p.exists():
        return None
    return pd.read_csv(p)


def reconstruct_and_run(short, long, prices):
    """Create MA strategy, generate signals on prices, run backtest and return result dict."""
    strat = MovingAverageStrategy(short_window=short, long_window=long)
    signals = strat.generate_signals(prices)
    bt = Backtester(initial_capital=10000.0, commission=0.0005, slippage=0.0002)
    res = bt.run(prices, signals, risk_per_trade=0.02, leverage=1.0)
    return res


def plot_equity_with_trades(result, label=None, ax=None):
    """Plot equity curve and mark entry/exit points from backtest result."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(10,4))
    eq = result.get('equity_curve', [])
    ax.plot(eq, label=label)

    # mark trades
    trades = result.get('trades', [])
    entries = [t for t in trades if t.get('type') == 'entry']
    exits = [t for t in trades if t.get('type') == 'exit']
    for e in entries:
        idx = e.get('index')
        price = e.get('price')
        if idx is not None and idx < len(eq):
            ax.scatter(idx, eq[idx], marker='^', color='green')
    for ex in exits:
        idx = ex.get('index')
        if idx is not None and idx < len(eq):
            ax.scatter(idx, eq[idx], marker='v', color='red')
    if label:
        ax.legend()
    return ax


In [ ]:
# Load existing wf results or generate sample
root = Path('.')
wf_json = root / 'wf_results.json'
wf_top_csv = root / 'wf_top5.csv'

wf = load_walkforward_json(wf_json)
top_df = load_top_csv(wf_top_csv)

if wf is None or top_df is None:
    print('wf_results.json or wf_top5.csv not found; generating sample data...')
    prices = [100 + i*0.3 for i in range(400)]
    shorts = [3,5,7]
    longs = [20,30]
    results = grid_search_ma(prices, shorts, longs, initial_capital=10000, commission=0.0005, slippage=0.0002)
    top = get_top_results(results, top_n=5, sort_key='sharpe')
    # build sample wf structure
    wf = [{'short': s, 'long': l, 'metrics': res} for s, l, res in results]
    top_df = pd.DataFrame([{'short': s, 'long': l, 'sharpe': res.get('sharpe'), 'total_return': res.get('total_return')} for s, l, res in results[:5]])
    print('Sample data generated')
else:
    print('Loaded wf_results.json and wf_top5.csv')

# Prepare full price series to inspect
if 'prices' not in locals():
    # try to reconstruct prices from wf if possible - fallback to synthetic
    prices = [100 + i*0.25 for i in range(400)]

print('Ready. Number of top rows:', len(top_df))


In [ ]:
# Plot top-3 equity curves and annotate trades

top_n = min(3, len(top_df))
fig, ax = plt.subplots(figsize=(12,6))
for i in range(top_n):
    row = top_df.iloc[i]
    s = int(row['short'])
    l = int(row['long'])
    label = f"MA {s}/{l}"
    res = reconstruct_and_run(s, l, prices)
    plot_equity_with_trades(res, label=label, ax=ax)

ax.set_title('Top strategies equity curves')
ax.set_xlabel('Period')
ax.set_ylabel('Equity')
fig.tight_layout()
out_path = Path('analysis/inspect_top_equity.png')
fig.savefig(out_path)
print('Saved', out_path)
plt.show()


## Prochaines étapes et conseils

- Ajuster les paramètres du backtester (commission, slippage, risk_per_trade) pour refléter votre réalité de trading.
- Pour analyser plus en profondeur, sauvegarder les equity curves des top-N dans des fichiers CSV et les réutiliser dans un notebook d'analyse.
- Utiliser `scripts/run_grid_top.sh` ou `scripts/run_walkforward_top.sh` pour générer des résultats sur jeux de données plus grands.
- Remplacer les prix synthétiques par des séries OHLC réelles et passer `highs`/`lows` au backtester pour un ATR plus précis.

Bonne exploration !

In [ ]:
# Auto-load top-5 (if present) and reconstruct top-3 equity curves, save CSVs and a combined PNG
import os
from pathlib import Path
import pandas as pd

wf_top = Path('wf_top5.csv')
if wf_top.exists():
    df = pd.read_csv(wf_top)
    top3 = df.head(3)
else:
    # fallback: use generated `top_df` from earlier cells if present
    try:
        top3 = top_df.head(3)
    except NameError:
        print('No wf_top5.csv or top_df available in notebook; please run the walk-forward script first.')
        top3 = None

if top3 is not None and not top3.empty:
    out_dir = Path('analysis')
    out_dir.mkdir(exist_ok=True)

    combined_fig_path = out_dir / 'inspect_top_top3.png'
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(10, 6))

    for i, row in top3.iterrows():
        short = int(row['short'])
        long = int(row['long'])
        label = f's={short}_l={long}'
        # reconstruct and run using helper from earlier cells
        try:
            res = reconstruct_and_run(short, long, prices)
        except NameError:
            # try to load synthetic prices used by script
            prices_local = [100 + i*0.3 for i in range(500)]
            res = reconstruct_and_run(short, long, prices_local)

        equity = res.get('equity_curve') or res.get('equity') or res.get('equity_curve', [])
        if equity is None:
            print(f'No equity for {label}')
            continue

        # save per-top equity CSV
        eq_df = pd.DataFrame({'equity': equity})
        eq_csv = out_dir / f'top_{i+1}_equity.csv'
        eq_df.to_csv(eq_csv, index=False)

        ax.plot(eq_df['equity'], label=label)

    ax.set_title('Top-3 equity curves (walk-forward)')
    ax.set_xlabel('Step')
    ax.set_ylabel('Equity')
    ax.legend()
    fig.tight_layout()
    fig.savefig(combined_fig_path)
    print(f'Saved top-3 equity CSVs to {out_dir} and combined figure to {combined_fig_path}')
else:
    print('No top combos to process.')